In [30]:
# importing dependencies
import pandas as pd
import glob
import os
import requests
from config import API_KEY

In [31]:
# setting path
files = os.path.join("Resources/RAW/Schools/greatschools*.csv")

In [32]:
#list of merged files returned
files = glob.glob(files)

In [33]:
# joining files 
school_df = pd.concat(map(pd.read_csv, files), ignore_index=True)
school_df.head(2)

,circle-rating--small,name,address,tablescraper-selected-row 2,tablescraper-selected-row 3,tablescraper-selected-row 4,tablescraper-selected-row href,rating-circle-small,scale,name href,tablescraper-selected-row,tablescraper-selected-row 6,tablescraper-selected-row href 2
0,10.0,Bird Rock Elementary School,"5371 La Jolla Hermosa Ave., La Jolla, CA, 92037",K-5,450,20:1,https://www.greatschools.org/california/la-jol...,NaN,NaN,NaN,NaN,NaN,NaN
1,9.0,Preuss School Ucsd,"9500 Gilman Drive, La Jolla, CA, 92093",6-12,836,20:1,https://www.greatschools.org/california/la-jol...,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
school_df.drop(['rating-circle-small','tablescraper-selected-row 2','tablescraper-selected-row 3','tablescraper-selected-row 4','tablescraper-selected-row href','tablescraper-selected-row 6','tablescraper-selected-row href 2','name href','tablescraper-selected-row'], axis=1, inplace=True)
school_df.dropna(axis=0, how='any', inplace=True)
school_df['Zip Code'] = school_df['address'].str[-5:]
school_df.reset_index(inplace=True)
school_df.drop(['index'],axis=1,inplace=True)
school_df

circle-rating--small          scale  \
0                     7.0  Above average   
1                     6.0        Average   
2                     6.0        Average   
3                     6.0        Average   
4                     4.0  Below average   
..                    ...            ...   
588                   3.0  Below average   
589                   3.0  Below average   
590                   3.0  Below average   
591                   2.0  Below average   
592                   1.0  Below average   

                                            name  \
0                            The Heights Charter   
1                 Boulder Oaks Elementary School   
2                    Joan Macqueen Middle School   
3                 Shadow Hills Elementary School   
4                       Alpine Elementary School   
..                                           ...   
588                     Bobier Elementary School   
589                 Rancho Minerva Middle School   
590  Vista Academy Of Visual And Performing Arts   
591     Major General Raymond Murray High School   
592        Alta Vista High (Continuation) School   

                                               address Zip Code  
0    2710 Alpine Boulevard, Suite E, Alpine, CA, 91901    91901  
1                  2320 Tavern Road, Alpine, CA, 91901    91901  
2                  2001 Tavern Road, Alpine, CA, 91901    91901  
3         8770 Harbison Canyon Road, Alpine, CA, 91901    91901  
4             1850 Alpine Boulevard, Alpine, CA, 91901    91901  
..                                                 ...      ...  
588            220 West Bobier Drive, Vista, CA, 92083    92083  
589              2245 Foothill Drive, Vista, CA, 92084    92084  
590        600 North Santa Fe Avenue, Vista, CA, 92083    92083  
591          215 North Melrose Drive, Vista, CA, 92083    92083  
592                 1575 Bonair Road, Vista, CA, 92084    92084  

[593 rows x 5 columns]

In [6]:
# link example format
# https://maps.googleapis.com/maps/api/geocode/json?address=1600+Amphitheatre+Parkway,+Mountain+View,+CA&key=YOUR_API_KEY
school_coords = []
key = API_KEY
addresses = school_df['address'].tolist()
for addy in addresses: 
    
    # cleaning addresses for URL
    zips = addy[-7:]
    new_addy = addy.strip(zips)
    new_addy = new_addy.replace(" ", "+")
    new_addy = new_addy.replace(",", "")
    
    # URL for google geocode API
    school_link = f"https://maps.googleapis.com/maps/api/geocode/json?address={new_addy}&key={key}"
    
    # Make a 'Get' request for the school location data.
    school_location = requests.get(school_link)
    
    # Get the JSON data.
    school_data = school_location.json()
    lat = school_data["results"][0]["geometry"]["location"]["lat"]
    lng = school_data["results"][0]["geometry"]["location"]["lng"]
    
#   print(new_addy)
#   print(school_link)
#   print(school_location)
#   print(lat)
#   print(lng)
#   print()

    school_coords.append({"lat": lat,
                         "lng": lng})

In [7]:
new_school_df = pd.DataFrame(school_coords, index=None, columns = ["lat", "lng"])
new_school_df

,lat,lng
0,32.835301,-116.755216
1,32.818612,-116.777682
2,32.819861,-116.778027
3,32.832502,-116.817095
4,32.843178,-116.798063
...,...,...
588,33.221216,-117.252026
589,33.206982,-117.208035
590,33.209944,-117.246714
591,33.195805,-117.254521


In [36]:
new_school_df['Zip Code'] = school_df['Zip Code']
new_school_df

,lat,lng,Zip Code
0,32.835301,-116.755216,91901
1,32.818612,-116.777682,91901
2,32.819861,-116.778027,91901
3,32.832502,-116.817095,91901
4,32.843178,-116.798063,91901
...,...,...,...
588,33.221216,-117.252026,92083
589,33.206982,-117.208035,92084
590,33.209944,-117.246714,92083
591,33.195805,-117.254521,92083


In [51]:
finaldf = school_df.merge(new_school_df, left_index=True, right_index=True)
finaldf

,circle-rating--small,scale,name,address,Zip Code_x,lat,lng,Zip Code_y
0,7.0,Above average,The Heights Charter,"2710 Alpine Boulevard, Suite E, Alpine, CA, 91901",91901,32.835301,-116.755216,91901
1,6.0,Average,Boulder Oaks Elementary School,"2320 Tavern Road, Alpine, CA, 91901",91901,32.818612,-116.777682,91901
2,6.0,Average,Joan Macqueen Middle School,"2001 Tavern Road, Alpine, CA, 91901",91901,32.819861,-116.778027,91901
3,6.0,Average,Shadow Hills Elementary School,"8770 Harbison Canyon Road, Alpine, CA, 91901",91901,32.832502,-116.817095,91901
4,4.0,Below average,Alpine Elementary School,"1850 Alpine Boulevard, Alpine, CA, 91901",91901,32.843178,-116.798063,91901
...,...,...,...,...,...,...,...,...
588,3.0,Below average,Bobier Elementary School,"220 West Bobier Drive, Vista, CA, 92083",92083,33.221216,-117.252026,92083
589,3.0,Below average,Rancho Minerva Middle School,"2245 Foothill Drive, Vista, CA, 92084",92084,33.206982,-117.208035,92084
590,3.0,Below average,Vista Academy Of Visual And Performing Arts,"600 North Santa Fe Avenue, Vista, CA, 92083",92083,33.209944,-117.246714,92083
591,2.0,Below average,Major General Raymond Murray High School,"215 North Melrose Drive, Vista, CA, 92083",92083,33.195805,-117.254521,92083


In [52]:
finaldf.drop(['Zip Code_y'], axis=1, inplace=True)
finaldf.rename(columns={'Zip Code_x':'zipcode'}, inplace=True)
finaldf['zipcode'] = finaldf['zipcode'].astype('int64')
finaldf

,circle-rating--small,scale,name,address,zipcode,lat,lng
0,7.0,Above average,The Heights Charter,"2710 Alpine Boulevard, Suite E, Alpine, CA, 91901",91901,32.835301,-116.755216
1,6.0,Average,Boulder Oaks Elementary School,"2320 Tavern Road, Alpine, CA, 91901",91901,32.818612,-116.777682
2,6.0,Average,Joan Macqueen Middle School,"2001 Tavern Road, Alpine, CA, 91901",91901,32.819861,-116.778027
3,6.0,Average,Shadow Hills Elementary School,"8770 Harbison Canyon Road, Alpine, CA, 91901",91901,32.832502,-116.817095
4,4.0,Below average,Alpine Elementary School,"1850 Alpine Boulevard, Alpine, CA, 91901",91901,32.843178,-116.798063
...,...,...,...,...,...,...,...
588,3.0,Below average,Bobier Elementary School,"220 West Bobier Drive, Vista, CA, 92083",92083,33.221216,-117.252026
589,3.0,Below average,Rancho Minerva Middle School,"2245 Foothill Drive, Vista, CA, 92084",92084,33.206982,-117.208035
590,3.0,Below average,Vista Academy Of Visual And Performing Arts,"600 North Santa Fe Avenue, Vista, CA, 92083",92083,33.209944,-117.246714
591,2.0,Below average,Major General Raymond Murray High School,"215 North Melrose Drive, Vista, CA, 92083",92083,33.195805,-117.254521


In [53]:
# get necessary zip
zipcodes_df = pd.read_csv("Resources/Clean/San_Diego_Housing_Data.csv")
zipcodes_df = zipcodes_df[['Zip Code']]
zipcodes_df.rename(columns = {'Zip Code':'zipcode'}, inplace = True)
new_df = finaldf[finaldf.zipcode.isin(zipcodes_df.zipcode)]
new_df

,circle-rating--small,scale,name,address,zipcode,lat,lng
0,7.0,Above average,The Heights Charter,"2710 Alpine Boulevard, Suite E, Alpine, CA, 91901",91901,32.835301,-116.755216
1,6.0,Average,Boulder Oaks Elementary School,"2320 Tavern Road, Alpine, CA, 91901",91901,32.818612,-116.777682
2,6.0,Average,Joan Macqueen Middle School,"2001 Tavern Road, Alpine, CA, 91901",91901,32.819861,-116.778027
3,6.0,Average,Shadow Hills Elementary School,"8770 Harbison Canyon Road, Alpine, CA, 91901",91901,32.832502,-116.817095
4,4.0,Below average,Alpine Elementary School,"1850 Alpine Boulevard, Alpine, CA, 91901",91901,32.843178,-116.798063
...,...,...,...,...,...,...,...
588,3.0,Below average,Bobier Elementary School,"220 West Bobier Drive, Vista, CA, 92083",92083,33.221216,-117.252026
589,3.0,Below average,Rancho Minerva Middle School,"2245 Foothill Drive, Vista, CA, 92084",92084,33.206982,-117.208035
590,3.0,Below average,Vista Academy Of Visual And Performing Arts,"600 North Santa Fe Avenue, Vista, CA, 92083",92083,33.209944,-117.246714
591,2.0,Below average,Major General Raymond Murray High School,"215 North Melrose Drive, Vista, CA, 92083",92083,33.195805,-117.254521


In [55]:
new_df.set_index('zipcode',inplace=True)
new_df.sort_index(inplace=True)
new_df

C:\Users\Adam\AppData\Local\Temp\ipykernel_4168\2481658283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.sort_index(inplace=True)


,circle-rating--small,scale,name,address,lat,lng
zipcode,,,,,,
91901,7.0,Above average,The Heights Charter,"2710 Alpine Boulevard, Suite E, Alpine, CA, 91901",32.835301,-116.755216
91901,6.0,Average,Boulder Oaks Elementary School,"2320 Tavern Road, Alpine, CA, 91901",32.818612,-116.777682
91901,6.0,Average,Joan Macqueen Middle School,"2001 Tavern Road, Alpine, CA, 91901",32.819861,-116.778027
91901,6.0,Average,Shadow Hills Elementary School,"8770 Harbison Canyon Road, Alpine, CA, 91901",32.832502,-116.817095
91901,4.0,Below average,Alpine Elementary School,"1850 Alpine Boulevard, Alpine, CA, 91901",32.843178,-116.798063
...,...,...,...,...,...,...
92173,4.0,Below average,Willow Elementary,"226 Willow Rd., San Ysidro, CA, 92173",32.552007,-117.042361
92173,4.0,Below average,La Mirada Elementary School,"222 Avenida de la Madrid, San Ysidro, CA, 92173",32.562873,-117.046277
92173,4.0,Below average,Smythe Elementary School,"1880 Smythe Ave., San Ysidro, CA, 92173",32.566630,-117.052827


In [56]:
from pathlib import Path  
filepath = Path('Resources/Clean/San_Diego_School_Data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
new_df.to_csv(filepath)